In [1]:
import numpy as np
import matplotlib.pyplot as py
import os
import cv2
from pathlib import Path
from imread import imread, imsave
import tifffile
import openslide
from PIL import Image

In [2]:
svs_images = Path('path/to/file')

In [ ]:
svs_images

In [4]:
test_svs = Path('path/to/file')

In [ ]:
test_svs

In [6]:
svs_stack = tifffile.imread(test_svs)

In [ ]:
svs_stack.shape

In [8]:
from PIL import Image, ImageFilter

def svs_to_png(svs_path, output_dir, target_size=(1024, 1024)):
    # Open the .svs file
    slide = openslide.open_slide(svs_path)
    
    # Read the entire slide region
    full_slide = slide.read_region((0, 0), 0, slide.level_dimensions[0])
    
    # Calculate resize dimensions while maintaining aspect ratio
    width_ratio = target_size[0] / full_slide.width
    height_ratio = target_size[1] / full_slide.height
    resize_ratio = min(width_ratio, height_ratio)
    new_width = int(full_slide.width * resize_ratio)
    new_height = int(full_slide.height * resize_ratio)
    
    # Resize the slide while maintaining aspect ratio
    resized_slide = full_slide.resize((new_width, new_height), Image.LANCZOS)
    
    # Create a blank canvas of the target size
    canvas = Image.new("RGB", target_size)
    
    # Calculate the position to paste the resized slide onto the canvas
    paste_position = ((target_size[0] - new_width) // 2, (target_size[1] - new_height) // 2)
    
    # Paste the resized slide onto the canvas
    canvas.paste(resized_slide, paste_position)
    
    # Define the output path for the PNG file
    png_path = os.path.join(output_dir, os.path.splitext(os.path.basename(svs_path))[0] + '.png')
    
    # Save the resized slide as a PNG file
    canvas.save(png_path, format='PNG')
    
    return png_path


In [9]:
def convert_directory(input_dir, output_dir):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Iterate over all files in the input directory
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.svs'):
                svs_path = os.path.join(root, file)
                png_path = svs_to_png(svs_path, output_dir)
                print(f"Converted {svs_path} to PNG format at {png_path}")

In [10]:
output_dir = 'path/to/output'

In [ ]:
convert_directory(svs_images, output_dir)